## UTILS

In [10]:
import numpy as np
import matplotlib as plt
import numpy.linalg as lng
from scipy.linalg import lu
from scipy import linalg
import networkx as nx
from scipy.stats.stats import pearsonr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Leer grafos

In [4]:
def leer_karateclub(path = "../datasets/karateclub_matriz.txt"):
  M = np.zeros(34*34).reshape(34, 34)
  with open(path,'r') as file:
    # reading each line 
    i = 0
    for line in file:
      line = line.split()
      for j in range(0, len(line)):

        M[i][j] = line[j]
      i+=1
  return M

In [9]:
def leer_labels_karate(path = "../datasets/karateclub_labels.txt"):
    file = open(path,'r')
    labels = np.zeros(34)
    i = 0
    with open(path,'r') as file:
    # reading each line    
        for line in file:
            labels[i] = line
            i+=1
    return labels

## Filtro facebook 

In [8]:
def leer_facebook():

    feat = pd.read_csv("../datasets/ego-facebook.feat", sep=" ",  header = None)
    edges_d = pd.read_csv("../datasets/ego-facebook.edges", sep=" ", header = None, names=["a", "b"])
    feat = feat.rename(columns={0: "a"})
    display(feat)
    #Filtro feat con los nodos de edges 
    edges = edges_d.values.flatten()
    edges = np.unique(edges).astype(int)
    edges =pd.DataFrame(edges)
    edges = edges.rename(columns ={0:"a"})

    feat = edges.merge(feat, on="a", how= "left")
    feat = feat.sort_values(by = ["a"])

    #saco los ind de los nodos 
    feat_sin_ind = feat
    feat_sin_ind.drop('a', axis = 1, inplace=True)

    F_att = feat_sin_ind.to_numpy() #mat de atributos 
    
    ed1 = edges_d["a"]
    ed2 = edges_d["b"]
    tags_ind = sorted(list(set(edges_d["a"])))
    f = np.array(np.zeros(786*786)).reshape(786, 786)
    for i in range(0, len(ed1)):
        ind_a = tags_ind.index(ed1[i])
        ind_b = tags_ind.index(ed2[i])
        f[ind_a][ind_b] = 1
        f[ind_b][ind_a] = 1

    return F_att, f


Leer grafos

In [7]:
def leer_grafo(path):
    file = open(path,'r')
    n = int(file.readline())
    m = int(file.readline())
    M = np.zeros((n, n))
    i = 0
    with open(path,'r') as file:
    # reading each line    
        for line in file:
            if(i>1):
                a = int(line.split()[0])
                b = int(line.split()[1])
                M[a-1][b-1] = 1
            i+=1
    return M

Generar archivos de los tests

In [ ]:
def generar_archivo_test_de_matriz(test_name, A):
    #Dada una matriz te genera el .txt en el formato requerido
    file = open('test_nuestros/' + test_name, 'w')
    
    with open('test_nuestros/' + test_name, 'w') as file:
        file.write(str(np.shape(A)[0])+"\n")
        file.write(str(int(np.sum(A))) + "\n")
        for i in range(0,np.shape(A)[0]):
            for j in range(0,np.shape(A)[0]):
                if(A[i][j] == 1):
                    file.write(str(i+1) + "\t" + str(j+1) +"\n")

Leer experimentos

In [ ]:
def read_results_time(test_name):
    times = []
    file = open("tiempos_exp/" + test_name + ".exp",'r')
    with open("tiempos_exp/" +test_name + ".exp",'r') as file:
        for line in file:
            times.append(float(line.split()[0]))
    return times

In [ ]:
def read_results_conv(test_name):
    iters = []
    epsilons = []
    file = open("iters_exp/" + test_name + ".exp",'r')
    with open("iters_exp/" +test_name + ".exp",'r') as file:
        for line in file:
            iters.append(int(line.split()[0]))
            epsilons.append(float(line.split()[1]))
    return iters, epsilons

In [ ]:
def results_times(test):
    
    timesEG = read_results_time(test + "EG")
    
    timesJac = read_results_time(test + "Jac")
    
    timesGS = read_results_time(test + "GS")

    """ print("Elim Gaussiana")
    print(np.var(timesEG))
    print(np.mean(timesEG))
    print("Jacobi")
    print(np.var(timesJac))
    print(np.mean(timesJac))
    print("Gauss-Seidel")
    print(np.var(timesGS))
    print(np.mean(timesGS)) """
    var = [np.var(timesEG), np.var(timesJac),np.var(timesGS)]
    mean = [np.mean(timesEG), np.mean(timesJac), np.mean(timesGS)]
    return mean, var
    

Calcular rankings

In [1]:
def calcular_ranks_de_matriz(G, p):
    n = np.shape(G)[0]
    cjs = grados(G)
    D = np.array(np.diag(cjs))
    I = np.array(np.eye((n)))
    R = G @ D
    pR = p*R
    A = I - pR
    col = np.array(np.ones(n))
    x = linalg.solve(A, col)
    normalized_x = (x)/np.sum(x)
    
    return normalized_x,np.mean(A) 

Metodos Iterativos

In [6]:
def jacobi(A,b,reps,x_ini,x_direct):
    # Matrices para los metodos iterativos
    D = np.diag(np.diag(A))
    L = np.triu(-A, k=1)
    U = np.tril(-A, k=-1)
    
    T =  lng.inv(D) @(L+U)
    c =  lng.inv(D) 

    print("Converge Jacobi: " , max(abs(np.linalg.eig(T)[0])) <1, max(abs(np.linalg.eig(T)[0])) ) 

    xi = x_ini 
    error = []
    for i in range(reps):
        xi = T@xi + c@b
        error.append(np.linalg.norm(xi - x_direct))
    return xi, error

In [5]:
def gauss_seidel(A,b,reps,x_ini,x_direct):
    # Matrices para los metodos iterativos
    D = np.diag(np.diag(A))

    L = np.tril(-A, k=-1)
    U = np.triu(-A, k=1) 

    T = (np.linalg.inv(D-L))@U 
    c = (np.linalg.inv(D-L))

    print("Converge GS: " , max(abs(np.linalg.eig(T)[0])) <1,  max(abs(np.linalg.eig(T)[0]))) 

    xi = x_ini 
    error = []
    for i in range(reps):
        xi = T@xi + c@b 
        error.append(np.linalg.norm(xi - x_direct))
    return xi, error

Metodos Directos

In [4]:
def grados(A):
    res = np.zeros(np.shape(A)[0])
    for i in range(0, np.shape(A)[0]):
        if(np.sum(A,axis = 0)[i] != 0):
            res[i] = 1/(np.sum(A,axis = 0)[i])
    return res
    
def calcular_ranks(test, p):
    G = leer_grafo( test)
    n = np.shape(G)[0]
    cjs = grados(G)
    D = np.array(np.diag(cjs))
    I = np.array(np.eye((n)))
    R = G @ D
    pR = p*R
    A = I - pR
    col = np.array(np.ones(n))
    x = lng.solve(A, col)
    normalized_x = (x)/np.sum(x)
    
    return normalized_x, np.mean(A) 

Armar grafos

In [ ]:
def armar_grafo_supernodo(n):
  #El nodo 0 es la página supernodo que es citada por todas y los demás nodos  
    supernodo = np.array(np.zeros(n*n)).reshape(n, n) ## i en w[i][j] j->i
    for j in range(1, n):
          # todos linkean al supernodo
          supernodo[0][j] = 1
    generar_archivo_test_de_matriz('test_supernodo_' + str(n) + '.txt', supernodo)
    return supernodo

In [ ]:
def experimento_supernodo_le_agrego_ejes(n, p):
    #Generar el supernodo vacio y calcula los rankings a medida de que 
    #Agrega links entrantes al supernodo. 
    
    W = np.array(np.zeros(n*n)).reshape(n,n)

    ranking_prom = np.array(np.zeros(n-1))
    pos_ranking_prom = np.array(np.zeros(n-1))
    
    for i in range(1, n):
        W[0][i] = 1
        res = calcular_ranks_de_matriz(W, p) 
        ranking_prom[i-1] = res[0]
        res_sort = -np.sort(-res)
        pos_ranking_prom[i-1] = np.where(res_sort == res[0])[0][0] + 1
    return [ranking_prom, pos_ranking_prom]

In [ ]:
def armar_grafo_antisupernodo(n):
    #El nodo 0 es la página antisupernodo que cita a todas pero nadie la cita.  
    antisupernodo = np.array(np.zeros(n*n)).reshape(n, n) ## i en w[i][j] j->i
    for j in range(1, n):
      # antisupernodo linkea a todos
      antisupernodo[j][0] = 1
    generar_archivo_test_de_matriz('test_antisupernodo_' + str(n) + '.txt', antisupernodo)
    return antisupernodo

In [ ]:
def experimento_antisupernodo_le_agrego_ejes(n, p):
    #Generar el anti-supernodo vacio y calcula los rankings a medida de que 
    #Agrega links salientes del anti-supernodo. 
    
    W = np.array(np.zeros(n*n)).reshape(n,n)

    ranking_prom = np.array(np.zeros(n-1))
    pos_ranking_prom = np.array(np.zeros(n-1))
    
    for i in range(1, n):
        W[i][0] = 1
        res = calcular_ranks_de_matriz(W, p) 
        ranking_prom[i-1] = res[0]
        res_sort = -np.sort(-res)
        pos_ranking_prom[i-1] = np.where(res_sort == res[0])[0][0] + 1
    return [ranking_prom, pos_ranking_prom]